## Initialization  

In [1]:
%load_ext autoreload
%autoreload 2
import os 
import sys
sys.path.insert(0, './src')
import time
import argparse
import yaml
import types
import copy, pprint
from time import sleep
from datetime import datetime
import numpy  as np
import torch  
import wandb
import pandas as pd
from utils.notebook_modules import initialize, init_dataloaders, init_environment, init_wandb, \
                                   training_prep, disp_dataloader_info,disp_info_1, \
                                   warmup_phase, weight_policy_training, disp_gpu_info

from utils.util import (print_separator, print_heading, timestring, print_loss) #, print_underline, load_from_pickle,
#                       print_dbg, get_command_line_args ) 

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=6, linewidth=132)
pd.options.display.width = 132
# torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=None)
# sys.path.insert(0, '/home/kbardool/kusanagi/AdaSparseChem/src')
# print(sys.path)
# disp_gpu_info() 
os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Training.ipynb"


## Create Environment

### Parse Input Args  - Read YAML config file

In [2]:
# RESUME_MODEL_CKPT = 'model_train_ep_25_seed_0088'

## For RESTARTING
##
# input_args = " --config yamls/chembl_3task_train.yaml " \
#              " --resume " \
#              " --exp_id      330i85cg" \
#              " --exp_name    0308_1204" \
#              " --exp_desc    Train with dropout 0.5" \
#              " --seed_idx    0 "\
#              " --batch_size  128" \
#              " --lambda_sparsity  0.01"\
#              " --lambda_sharing   0.01" 
## get command line arguments

In [3]:
##  For Initiating 
##
input_args = " --config yamls/chembl_3task_train.yaml " \
             " --exp_desc    6 lyrs,dropout 0.5, weight 105 bch/ep policy 105 bch/ep " \
             " --hidden_size   150 150 150 150 150 150   " \
             " --tail_hidden_size   150" \
             " --seed_idx            0" \
             " --batch_size        128" \
             " --task_lr          0.001" \
             " --backbone_lr      0.001" \
             " --policy_lr        0.01" \
             " --lambda_sparsity  0.03" \
             " --lambda_sharing   0.01" 

#              " --hidden_size   100 100 100 100 100 100" \
#              " --tail_hidden_size  100 " \
                         

In [4]:
opt, ns = initialize(input_args, build_folders = True)


  command line parms : 
------------------------
 config...................  yamls/chembl_3task_train.yaml
 exp_id...................  None
 exp_name.................  None
 folder_sfx...............  None
 exp_desc.................  6 lyrs,dropout 0.5, weight 105 bch/ep policy 105 bch/ep
 hidden_sizes.............  [150, 150, 150, 150, 150, 150]
 tail_hidden_size.........  150
 seed_idx.................  0
 batch_size...............  128
 backbone_lr..............  0.001
 task_lr..................  0.001
 policy_lr................  0.01
 decay_lr_rate............  None
 decay_lr_freq............  None
 lambda_sparsity..........  0.03
 lambda_sharing...........  0.01
 gpu_ids..................  [0]
 resume...................  False
 cpu......................  False



##################################################
################### READ YAML ####################
##################################################


 log_dir              create folder:  ../experiments/AdaSparseChe

### Setup Dataloader and Model  

In [5]:
dldrs = init_dataloaders(opt)

disp_dataloader_info(dldrs)

environ = init_environment(ns, opt, is_train = True, policy_learning = False, display_cfg = False)

# ********************************************************************
# **************** define optimizer and schedulers *******************
# ********************************************************************                                
environ.define_optimizer(policy_learning=False)
environ.define_scheduler(policy_learning=False)


##################################################
############### CREATE DATALOADERS ###############
##################################################

 trainset.y_class                                   :  [(13331, 5), (13331, 5), (13331, 5)] 
 trainset1.y_class                                  :  [(13331, 5), (13331, 5), (13331, 5)] 
 trainset2.y_class                                  :  [(13331, 5), (13331, 5), (13331, 5)] 
 valset.y_class                                     :  [(4137, 5), (4137, 5), (4137, 5)]  
 testset.y_class                                    :  [(920, 5), (920, 5), (920, 5)]  
                                 
 size of training set 0 (warm up)                   :  13331 
 size of training set 1 (network parms)             :  13331 
 size of training set 2 (policy weights)            :  13331 
 size of validation set                             :  4137 
 size of test set                                   :  920 
                               Total           

In [6]:
print(f" Current LR: {environ.optimizers['alphas'].param_groups[0]['lr'] }")
print(f" Current LR: {environ.optimizers['weights'].param_groups[0]['lr']}")
print(f" Current LR: {environ.optimizers['weights'].param_groups[1]['lr']}")

 Current LR: 0.01
 Current LR: 0.001
 Current LR: 0.001


In [7]:
environ.optimizers['weights'].param_groups[0]

{'params': [Parameter containing:
  tensor([[-0.039199, -0.026896,  0.015696,  0.021018, -0.059028, -0.069487,  0.080026, -0.056681, -0.030938, -0.002778, -0.044748,
           -0.045918,  0.050549, -0.007592,  0.062252,  0.069579,  0.070078, -0.059879, -0.039095, -0.014477, -0.044603, -0.005471,
           -0.007979,  0.061740, -0.011913, -0.033068, -0.045195,  0.048141, -0.052791, -0.053301, -0.031594, -0.055418,  0.068755,
            0.066495,  0.022349, -0.073826, -0.070488,  0.055380, -0.035242, -0.051436, -0.033102,  0.021591,  0.002433, -0.058409,
           -0.067490,  0.052191,  0.049606,  0.080167, -0.030325,  0.017604, -0.036159,  0.000493,  0.012687, -0.063100, -0.050174,
            0.028348,  0.077406,  0.003578, -0.057715, -0.013247,  0.013125, -0.007682, -0.010720,  0.048879, -0.077897,  0.074210,
            0.035563, -0.036086,  0.035048, -0.032139,  0.058121, -0.018512,  0.068136,  0.060880,  0.065124, -0.050969,  0.052900,
            0.034117,  0.076649,  0.075088

###  Weights and Biases Initialization 

In [8]:
init_wandb(ns, opt, environment = environ)

print(f" PROJECT NAME: {ns.wandb_run.project}\n"
      f" RUN ID      : {ns.wandb_run.id} \n"
      f" RUN NAME    : {ns.wandb_run.name}") 

wandb: Currently logged in as: kbardool (use `wandb login --relogin` to force relogin)


1oii7m2w 0325_1044 AdaSparseChem


 PROJECT NAME: AdaSparseChem
 RUN ID      : 1oii7m2w 
 RUN NAME    : 0325_1044
 PROJECT NAME: AdaSparseChem
 RUN ID      : 1oii7m2w 
 RUN NAME    : 0325_1044


In [9]:
# ns.wandb_run.finish()

### Initiate / Resume Training Prep

In [10]:
if opt['train']['resume']:
    print(opt['train']['which_iter'])
    print(opt['paths']['checkpoint_dir'])
    print(RESUME_MODEL_CKPT)
    # opt['train']['resume'] = True
    # opt['train']['which_iter'] = 'warmup_ep_40_seed_0088'
if opt['train']['resume']:
    print_separator('Resume training')
    loaded_iter, loaded_epoch = environ.load_checkpoint(RESUME_MODEL_CKPT, path = opt['paths']['checkpoint_dir'], verbose = True)
    print(loaded_iter, loaded_epoch)    
#     current_iter = environ.load_checkpoint(opt['train']['which_iter'])
    environ.networks['mtl-net'].reset_logits()
    val_metrics = load_from_pickle(opt['paths']['checkpoint_dir'], RESUME_METRICS_CKPT)
    # training_prep(ns, opt, environ, dldrs, epoch = loaded_epoch, iter = loaded_iter )

else:
    print_separator('Initiate Training ')

##################################################
############### Initiate Training  ###############
##################################################


### Training Preparation

In [11]:
training_prep(ns, opt, environ, dldrs )

 cuda available [0]
 set print_freq to length of train loader: 105
 set eval_iters to length of val loader  : 33


In [12]:
disp_info_1(ns, opt, environ)


 Num_blocks                : 6                                

 batch size                : 128 
 batches/ Weight trn epoch : 105 
 batches/ Policy trn epoch : 105                                 

 Print Frequency           : -1 
 Config Val Frequency      : 500 
 Config Val Iterations     : -1 
 Val iterations            : 33 
 which_iter                : warmup 
 train_resume              : False                                 
 
 fix BN parms              : False 
 Task LR                   : 0.001 
 Backbone LR               : 0.001                                 

 Sharing  regularization   : 0.01 
 Sparsity regularization   : 0.03 
 Task     regularization   : 1                                 

 Current epoch             : 0  
 Warm-up epochs            : 75 
 Training epochs           : 125


In [13]:
print(environ.disp_for_excel())


    folder: 150x6_0325_1044_plr0.01_sp0.03_sh0.01_lr0.001
    layers: 6 [150, 150, 150, 150, 150, 150] 
    
    middle dropout         : 0.5
    last dropout           : 0.5
    diff_sparsity_weights  : False
    skip_layer             : 0
    is_curriculum          : False
    curriculum_speed       : 3
    
    task_lr                : 0.001
    backbone_lr            : 0.001
    decay_lr_rate          : 0.75
    decay_lr_freq          : 40
    
    policy_lr              : 0.01
    policy_decay_lr_rate   : 0.75
    policy_decay_lr_freq   : 50
    lambda_sparsity        : 0.03
    lambda_sharing         : 0.01
    lambda_tasks           : 1
    
    Gumbel init_temp       : 4
    Gumbel decay_temp      : 0.965
    Gumbel decay_temp_freq : 16
    Logit init_method      : random
    Logit init_neg_logits  : None
    Logit hard_sampling    : False
    Warm-up epochs         : 75
    training epochs        : 125
    Data split ratios      : [0.725, 0.225, 0.05]



## Warmup Training

In [14]:
# environ.display_trained_policy(ns.current_epoch,out=sys.stdout)
# ns.stop_epoch_warmup = 10
# ns.warmup_epochs = 10
print(ns.warmup_epochs, ns.current_epoch)
print_heading(f" Last Epoch: {ns.current_epoch}   # of warm-up epochs to do:  {ns.warmup_epochs} - Run epochs {ns.current_epoch+1} to {ns.current_epoch + ns.warmup_epochs}", verbose = True)

75 0
------------------------------------------------------------------------
 Last Epoch: 0   # of warm-up epochs to do:  75 - Run epochs 1 to 75
------------------------------------------------------------------------ 



In [15]:
# warmup_phase(ns,opt, environ, dldrs, epochs = 25)
warmup_phase(ns,opt, environ, dldrs)

------------------------------------------------------------------------
 Last Epoch: 0   # of warm-up epochs to do:  75 - Run epochs 1 to 75
------------------------------------------------------------------------ 

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
    1 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |   10.3443   6.2375e-02   1.8974e-04   10.4069 |   0.68862   0.57713   0.58059   0.57676 |   10.3291   6.2375e-02   1.8974e-04    10.3917 |   9.3 |
    2 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    9.8342   6.2375e-02   1.8974e-04    9.8967 |   0.68006   0.61722   0.62208   0.61689 |   10.2011   6.2375e-02   1.8974e-04    10.2637 |   9.9 |
    3 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    9.8170   6.2375e-02   1.8974e-04    9.8795 |   0.67029   0.63643   0.64230   0.63612 |   10.0561   6.2375e-02   1.8974e-04 

   39 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    4.8681   6.2375e-02   1.8974e-04    4.9307 |   0.53333   0.81173   0.81285   0.81164 |    7.9961   6.2375e-02   1.8974e-04     8.0586 |   9.7 |
   40 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    4.7278   6.2375e-02   1.8974e-04    4.7904 |   0.53278   0.81354   0.81405   0.81345 |    7.9919   6.2375e-02   1.8974e-04     8.0545 |  10.2 |
   41 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    3.2963   6.2375e-02   1.8974e-04    3.3589 |   0.53259   0.81279   0.81431   0.81269 |    7.9821   6.2375e-02   1.8974e-04     8.0447 |   9.5 |
   42 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    5.0915   6.2375e-02   1.8974e-04    5.1541 |   0.53672   0.81416   0.81482   0.81406 |    8.0455   6.2375e-02   1.8974e-04     8.1080 |  11.1 |
   43 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    4.5429   6.2375e-02   1.8974e-04    4.6055 |   0.53364   0.81394   0.81395   0.81385 |    7.9934   6.2375e-02   1.8974e-04     8.0560 |  10

In [18]:
# warmup_phase(ns,opt, environ, dldrs, epochs = 25)

In [19]:
# ns.wandb_run.finish()

In [20]:
# ns.wandb_run.finish()

In [21]:
# environ.losses

In [22]:
# environ.val_metrics

#### display parms

In [16]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")
print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #
print(' current lr: ', environ.optimizers['alphas'].param_groups[0]['lr'],)
print(' current lr: ', environ.optimizers['weights'].param_groups[0]['lr'])
print(' current lr: ', environ.optimizers['weights'].param_groups[1]['lr'])

 Backbone Learning Rate      : 0.001
 Tasks    Learning Rate      : 0.001
 Policy   Learning Rate      : 0.01

 Sparsity regularization     : 0.03
 Sharing  regularization     : 0.01 

 Tasks    regularization     : 1   
 Gumbel Temp                 : 4.0000         
 Gumbel Temp decay           : 16
 current lr:  0.01
 current lr:  0.001
 current lr:  0.001


In [24]:
# environ.opt['train']['policy_lr'] = 0.01
# opt['train']['policy_lr']         = 0.01
# environ.opt['train']['lambda_sparsity'] = 0.1
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# environ.opt['train']['decay_temp_freq'] = 2
# print(environ.optimizers['alphas'].param_groups)
# print(environ.optimizers['weights'].param_groups)
# print('initial lr: ', environ.optimizers['alphas'].param_groups[0]['initial_lr'] , 'current lr: ', environ.optimizers['alphas'].param_groups[0]['lr'],)
# print('current lr: ', environ.optimizers['weights'].param_groups[0]['lr'])
# print('current lr: ', environ.optimizers['weights'].param_groups[1]['lr'])

## Weight & Policy Training

### Weight/Policy Training Preparation

In [25]:
# ns.flag_warmup = True

In [17]:
if ns.flag_warmup:
    print_heading( f"** {timestring()} \n"
                   f"** Training epoch: {ns.current_epoch} iter: {ns.current_iter}   flag: {ns.flag} \n"
                   f"** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)\n"
                   f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
                   f"** Take checkpoint and block gradient flow through Policy net", verbose=True)
#     environ.define_optimizer(policy_learning=True)
#     environ.define_scheduler(policy_learning=True)
    ns.flag_warmup = False
    ns.flag = 'update_weights'
    environ.fix_alpha()
    environ.free_weights(opt['fix_BN'])

------------------------------------------------------------------------------------------------------------------------
** 2022-03-25 12:39:41:997292 
** Training epoch: 75 iter: 7875   flag: update_weights 
** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)
** Switch from Warm Up training to Alternate training Weights & Policy 
** Take checkpoint and block gradient flow through Policy net
------------------------------------------------------------------------------------------------------------------------ 



In [18]:
# num_train_layers = None 
# environ.opt['is_curriculum'] = True
# environ.opt['curriculum_speed'] = 4
# ns.num_train_layers = None

In [19]:
print(ns.current_epoch,ns.training_epochs)
print(f"ns.current_epoch           : {ns.current_epoch}") 
print(f"ns.current_iters           : {ns.current_iter} \n")  
print(f"ns.training_epochs         : {ns.training_epochs}") 
# print(f"ns.stop_epoch_training     : {ns.stop_epoch_training}")
print(f"Batches in weight epoch    : {ns.stop_iter_w}")
print(f"Batches in policy epoch    : {ns.stop_iter_a}")
print(f"num_train_layers           : {ns.num_train_layers}")
print()

75 125
ns.current_epoch           : 75
ns.current_iters           : 7875 

ns.training_epochs         : 125
Batches in weight epoch    : 105
Batches in policy epoch    : 105
num_train_layers           : None



In [20]:
ns.training_epochs = 250
print_loss(environ.val_metrics, title = f"[e] Last ep:{ns.current_epoch}  it:{ns.current_iter}")
# environ.display_trained_policy(ns.current_epoch)
# environ.display_trained_logits(ns.current_epoch)

[e] Last ep:75  it:7875 -  Total Loss: 8.5676     
Task: 8.5051   Sparsity: 6.23750e-02    Sharing: 1.89744e-04 


In [21]:
# ns.current_epoch = 200
# ns.flag 

In [22]:
print_heading(f" Last Epoch Completed : {ns.current_epoch}       # of epochs to run:  {ns.training_epochs} -->  epochs {ns.current_epoch+1} to {ns.training_epochs + ns.current_epoch}"
              f"\n policy_learning rate : {environ.opt['train']['policy_lr']} "
              f"\n lambda_sparsity      : {environ.opt['train']['lambda_sparsity']}"
              f"\n lambda_sharing       : {environ.opt['train']['lambda_sharing']}"
              f"\n curriculum training  : {opt['is_curriculum']}     cirriculum speed: {opt['curriculum_speed']}     num_training_layers : {ns.num_train_layers}", 
              verbose = True)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 75       # of epochs to run:  250 -->  epochs 76 to 325
 policy_learning rate : 0.01 
 lambda_sparsity      : 0.03
 lambda_sharing       : 0.01
 curriculum training  : False     cirriculum speed: 3     num_training_layers : None
------------------------------------------------------------------------------------------------------------------------ 



### Weight/Policy Training

In [ ]:
# weight_policy_training(ns, opt, environ, dldrs, epochs = 100)
weight_policy_training(ns, opt, environ, dldrs)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 75   # of epochs to run:  250 -->  epochs 76 to 325    
 policy_learning rate : 0.01      
 lambda_sparsity      : 0.03
 lambda_sharing       : 0.01 
 curriculum training  : False     cirriculum speed: 3     num_training_layers : None
------------------------------------------------------------------------------------------------------------------------ 

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
   76 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    2.9032   6.2375e-02   1.8974e-04    2.9658 |   0.59069   0.80070   0.80344   0.80060 |    8.8564   6.2375e-02   1.8974e-04     8.9189 |  13.4 |
Previous best_epoch:     0   best iter:     0,   best_value: 0.00000
New      best_epoch:    76

   87 |   7.50e-04   7.50e-04   1.00e-02  4.000e+00 |    2.5133   7.8480e-02   1.9979e-04    2.5920 |   0.64675   0.80878   0.81156   0.80867 |    9.7281   7.8529e-02   2.6835e-04     9.8069 |  10.6 |
   88 |   7.50e-04   7.50e-04   1.00e-02  4.000e+00 |    2.5355   7.8529e-02   2.6835e-04    2.6143 |   0.64162   0.80849   0.81182   0.80840 |    9.6366   7.8529e-02   2.6835e-04     9.7154 |  14.4 |
   88 |   7.50e-04   7.50e-04   1.00e-02  4.000e+00 |    2.2297   8.3823e-02   5.2425e-04    2.3140 |   0.67408   0.80787   0.81009   0.80778 |   10.0769   8.3970e-02   4.5137e-04    10.1614 |  10.9 |
   89 |   7.50e-04   7.50e-04   1.00e-02  4.000e+00 |    2.3032   8.3970e-02   4.5137e-04    2.3876 |   0.66568   0.81229   0.81409   0.81219 |   10.0015   8.3970e-02   4.5137e-04    10.0859 |  14.8 |
   89 |   7.50e-04   7.50e-04   1.00e-02  4.000e+00 |    2.9681   8.1361e-02   3.5664e-04    3.0498 |   0.65547   0.81282   0.81370   0.81272 |    9.8153   8.1314e-02   3.8608e-04     9.8970 |  11

  101 |   7.50e-04   7.50e-04   1.00e-02  3.860e+00 |    2.1510   7.5024e-02   2.7501e-04    2.2263 |   0.68595   0.81178   0.81377   0.81169 |   10.2628   7.5129e-02   2.5132e-04    10.3382 |  11.6 |
  102 |   7.50e-04   7.50e-04   1.00e-02  3.860e+00 |    2.0833   7.5129e-02   2.5132e-04    2.1587 |   0.67866   0.80990   0.81273   0.80979 |   10.1576   7.5129e-02   2.5132e-04    10.2330 |  14.8 |
  102 |   7.50e-04   7.50e-04   1.00e-02  3.860e+00 |    2.0288   6.9265e-02   3.7653e-04    2.0984 |   0.66689   0.81172   0.81374   0.81161 |   10.0140   6.9306e-02   3.3574e-04    10.0836 |  11.6 |
  103 |   7.50e-04   7.50e-04   1.00e-02  3.860e+00 |    2.0494   6.9306e-02   3.3574e-04    2.1191 |   0.66958   0.80775   0.81189   0.80763 |   10.0584   6.9306e-02   3.3574e-04    10.1280 |  14.6 |
  103 |   7.50e-04   7.50e-04   1.00e-02  3.860e+00 |    2.2328   6.9870e-02   3.3173e-04    2.3030 |   0.68997   0.81312   0.81426   0.81302 |   10.3281   6.9911e-02   3.4250e-04    10.3984 |  12

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
  116 |   7.50e-04   7.50e-04   1.00e-02  3.725e+00 |    1.9062   6.3793e-02   4.2945e-04    1.9704 |   0.69637   0.80863   0.81243   0.80853 |   10.4308   6.3793e-02   4.2945e-04    10.4950 |  14.8 |
  116 |   7.50e-04   7.50e-04   1.00e-02  3.725e+00 |    1.7491   6.1782e-02   4.6018e-04    1.8114 |   0.67536   0.80758   0.81153   0.80748 |   10.1220   6.1785e-02   3.9774e-04    10.1842 |  12.0 |
  117 |   7.50e-04   7.50e-04   1.00e-02  3.725e+00 |    2.0551   6.1785e-02   3.9774e-04    2.1173 |   0.69173   0.80728   0.81087   0.80717 |   10.3826   6.1785e-02   3.9774e-04    10.4448 |  15.6 |
  117 |   7.50e-04   7.50e-04   1.00e-02  3.725e+00 |    1.9108   6.2830e-02   2.4953e-04    1.9738 |   0.68504   0.81096   0.81403   0.81085 |   10.2709   6.2840e-02   2.2896e-04    10.3339 |  12

In [33]:
ns.best_epoch, ns.best_iter, ns.best_value

(90, 10920, 0.8168097033305215)

### Close WandB run

In [34]:
wandb.finish()

epoch,▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
train_time,▁▁▁▁▁▂▅▄▅▅▅▅▅▅▄▅▄▅▅▄█▅▅▅▇▆▆▅▅▅▅▆▅▄▅▄▄▄▄▄
epoch,325
train_time,10.22264


In [ ]:
# ns.best_epoch = 0
# from utils.notebook_modules import wrapup_phase
# wrapup_phase(ns, opt, environ)

In [ ]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")


print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #
print(opt['train']['decay_temp_freq'])  


In [ ]:
# environ.opt['train']['policy_lr']       = 0.002
# environ.opt['train']['lambda_sparsity'] = 0.05
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# # environ.opt['train']['decay_temp_freq'] = 2
 

In [ ]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")


print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

print()
print( f" current_iters               : {ns.current_iter}")  
print( f" current_epochs              : {ns.current_epoch}") 
print( f" train_total_epochs          : {ns.training_epochs}") 
print( f" stop_epoch_training         : {ns.stop_epoch_training}")

## Post Training Stuff

In [61]:
# pp.pprint(environ.losses)
# pp.pprint(environ.val_metrics)
environ.num_layers, environ.networks['mtl-net'].num_layers

(6, 6)

In [ ]:
# pp.pprint(environ.val_metrics['total'])

In [ ]:
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch)
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# model_label   = 'model_train_ep_%d_seed_%04d' % (current_epoch, opt['random_seed'])
# metrics_label = 'metrics_train_ep_%d_seed_%04d.pickle' % (current_epoch, opt['random_seed'])
# environ.save_checkpoint(model_label, current_iter, current_epoch) 
# save_to_pickle(environ.val_metrics, environ.opt['paths']['checkpoint_dir'], metrics_label)
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# print_loss(current_iter, environ.losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, trn_losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, environ.val_metrics, title = f"[e] Policy training epoch:{current_epoch}    iter:")

In [ ]:
# environ.losses
# environ.val_metrics

In [ ]:
# environ.batch_data
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, 0, out=[sys.stdout])
# environ.display_parameters()

# with np.printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', precision=7, formatter={'float': lambda x: f"{x:12.5e}"}):
#     environ.print_logit_grads('gradients')

# environ_params = environ.get_task_specific_parameters()
# environ_params = environ.get_arch_parameters()
# environ_params = environ.get_backbone_parameters()
# print(environ_params)
# for param in environ_params:
#     print(param.grad.shape, '\n', param.grad)
#     print(param)

In [ ]:
environ.display_trained_logits(ns.current_epoch)
environ.display_trained_policy(ns.current_epoch)

In [ ]:
environ.display_test_sample_policy(ns.current_epoch, hard_sampling = True)
environ.display_train_sample_policy(ns.current_epoch, hard_sampling = True)

In [ ]:
# environ.define_optimizer(policy_learning=True)

In [ ]:
print(environ.optimizers['alphas'])
print(environ.optimizers['weights'])

In [ ]:
print('Policy  initial_lr : ', environ.optimizers['alphas'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['alphas'].param_groups[0]['lr'])
print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[0]['lr'])
print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[1]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[1]['lr'])

In [ ]:
wandb.run is None

In [ ]:
# opt['exp_instance'] = '0218_1358'     
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"
# print()
# opt['exp_instance'] = datetime.now().strftime("%m%d_%H%M")
# opt['exp_description'] = f"No Alternating Weight/Policy - training all done with both weights and policy"
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"

In [ ]:
# wandb.finish()

In [ ]:
# 
p = environ.get_current_state(0)

In [ ]:
pp.pprint(p)

### Post Warm-up Training stuff

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
environ.networks['mtl-net'].arch_parameters()

In [ ]:
p = environ.get_sample_policy(hard_sampling = False)
print(p)
p = environ.get_policy_prob()
print(p)
p = environ.get_policy_logits()
print(p)

# p = environ.get_current_policy()
# print(p)

In [ ]:
a = softmax([0.0, 1])
print(a)
sampled = np.random.choice((1, 0), p=a)
print(sampled)

In [ ]:
print(environ.optimizers['weights'])
print(environ.schedulers['weights'].get_last_lr())

In [ ]:
print('losses.keys      : ', environ.losses.keys())
print('losses[task]keys : ', environ.losses['task1'].keys())
pp.pprint(environ.losses)

In [ ]:
print( environ.val_metrics.keys())
# pp.pprint(val_metrics)
print(type(environ.val_metrics['aggregated']))
print()
print(type(environ.val_metrics['task1']['classification_agg']))
print()
pp.pprint(environ.val_metrics)

In [ ]:
# import pickle
# with open("val_metrics.pkl", mode= 'wb') as f:
#         pickle.dump(val_metrics, f)
    
# with open('val_metrics.pkl', 'rb') as f:    
#     tst_val_metrics = pickle.load(f)

In [ ]:
# print(environ.input.shape) 
# a = getattr(environ, 'task1_pred')
# yc_data = environ.batch['task1_data']
# print(yc_data.shape)
# yc_ind = environ.batch['task1_ind']
# print(yc_ind.shape)
# yc_hat_all = getattr(environ, 'task1_pred')
# print(yc_hat_all.shape)
# yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
# print(yc_hat_all.shape, yc_hat.shape)

# 
# environ.losses
# loss = {}
# for key in environ.losses.keys():
#     loss[key] = {}
#     for subkey, v in environ.losses[key].items():
#         print(f" key:  {key}   subkey: {subkey} ")
#         if isinstance(v, torch.Tensor):
#             loss[key][subkey] = v.data
#             print(f" Tensor  -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
#         else:
#             loss[key][subkey] = v
#             print(f" integer -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
# pp.pprint(tst_val_metrics)             

In [ ]:
# print('metrics.keys: ', environ.metrics.keys())
# print('metrics[task].keys: ', environ.metrics['task1'].keys())
# pp.pprint(environ.metrics['task1'])
# pp.pprint(environ.losses['task1']['total'])

In [ ]:
# title='Iteration'
# for t_id, _ in enumerate(environ.tasks):
#     task_key = f"task{t_id+1}"
# #     print_heading(f"{title}  {current_iter}  {task_key} : {val_metrics[task_key]['classification_agg']}", verbose = True)

#     for key, _  in val_metrics[task_key]['classification_agg'].items():
#         print('%s/%-20s'%(task_key, key), val_metrics[task_key]['classification_agg'][key], current_iter)
#         print(f"{task_key:s}/{key:20s}", val_metrics[task_key]['classification_agg'][key], current_iter)
#         print()
#             # print_current_errors(os.path.join(self.log_dir, 'loss.txt'), current_iter,key, loss[key], time.time() - start_time)

In [ ]:
# environ.print_loss(current_iter, start_time, metrics = val_metrics['loss'], verbose=True)
# print(opt['lambdas'])
# p = (opt['lambdas'][0] * environ.losses['tasks']['task1'])
# print(p)

# environ.print_val_metrics(current_iter, start_time, val_metrics , title='validation', verbose=True)    

In [ ]:
# print(current_iter)
# print_metrics_cr(current_iter, t1 - t0, None, val_metrics , True)
# environ.print_val_metrics(current_iter, start_time, val_metrics, title='validation', verbose = True)

In [ ]:
print(f" val_metric keys               : {val_metrics.keys()}")
print(f" loss keys                     : {val_metrics['loss'].keys()}")
print(f" task1 keys                    : {val_metrics['task1'].keys()}")
print(f" task1 classification keys     : {val_metrics['task1']['classification'].keys()}")
print(f" task1 classification_agg keys : {val_metrics['task1']['classification_agg'].keys()}")
print()
print(f" task1                       : {val_metrics['task1']['classification_agg']['loss']:5f}")
print(f" task2                       : {val_metrics['task2']['classification_agg']['loss']:5f}")
print(f" task3                       : {val_metrics['task3']['classification_agg']['loss']:5f}")
print(f" loss                        : {val_metrics['loss']['total']:5f}")
print(f" train_time                  : {val_metrics['train_time']:2f}")
print(f" epoch                       : {val_metrics['epoch']}")


### Post Weight + Policy Training Stuff 

In [ ]:
environ.networks['mtl-net'].backbone.layer_config

In [ ]:
num_blocks = 6
num_policy_layers = 6
gt =  torch.ones((num_blocks)).long()
gt0 =  torch.zeros((num_blocks)).long()
print(gt)
print(gt0)

loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
print(loss_weights)

In [ ]:
if environ.opt['diff_sparsity_weights'] and not environ.opt['is_sharing']:
    print(' cond 1')
    ## Assign higher weights to higher layers 
    loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
    print(f"{task_key} sparsity error:  {2 * (loss_weights[-num_blocks:] * environ.cross_entropy2(logits[-num_blocks:], gt)).mean()})")
    print_dbg(f" loss_weights :  {loss_weights}", verbose = True)
    print_dbg(f" cross_entropy:  {environ.cross_entropy2(logits[-num_blocks:], gt)}  ", verbose = True)
    print_dbg(f" loss[sparsity][{task_key}]: {self.losses['sparsity'][task_key] } ", verbose = True)

else:
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between \n Logits   : \n{logits[-num_blocks:]} \n and gt: \n{gt} \n", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-num_blocks:], gt)}")
    
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt[-1:])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt0[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt0[-1:])} \n")
    
    print('\n cond 3')    
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt[0:1])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt0[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt0[0:1])} \n")
        
        

In [ ]:
# flag = 'update_w'
# environ.fix_alpha
# environ.free_w(opt['fix_BN'])

flag = 'update_alpha'
environ.fix_weights()
environ.free_alpha()

In [ ]:
environ.networks['mtl-net'].num_layers

In [ ]:
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

train_total_epochs += 5

print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

In [ ]:
# print_metrics_cr(current_epoch, time.time() - t0, None, environ.val_metrics , num_prints)      

# num_prints += 1
# t0 = time.time()

# # Take check point
# environ.save_checkpoint('latest', current_iter)
# environ.train()
# #-------------------------------------------------------
# # END validation process
# #-------------------------------------------------------       
# flag = 'update_alpha'
# environ.fix_w()
# environ.free_alpha()

In [ ]:
# dilation = 2
# kernel_size = np.asarray((3, 3))
# upsampled_kernel_size = (kernel_size - 1) * (dilation - 1) + kernel_size
# print(upsampled_kernel_size)

In [ ]:
# environ.optimizers['weights'].param_groups[0]
# for param_group in optimizer.param_groups:
#     return param_group['lr']

In [ ]:
environ.schedulers['weights'].get_last_lr()

In [ ]:
current_state = {}
for k, v in environ.optimizers.items():
    print(f'state dict for {k} = {v}')
    current_state[k] = v.state_dict()
pp.pprint(current_state)

In [ ]:
current_state = {}
for k, v in environ.schedulers.items():
    print(f'state dict for {k} = {v}')
    print(v.state_dict())

### Losses and Metrics

In [ ]:
trn_losses = environ.losses

In [ ]:
print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# pp.pprint(environ.losses)
pp.pprint(trn_losses)

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
# environ.opt['train']['Lambda_sharing'] = 0.5
# opt['train']['Lambda_sharing'] = 0.5

# environ.opt['train']['policy_lr'] = 0.001
# opt['train']['policy_lr'] = 0.001

In [ ]:
environ.losses.keys()
pp.pprint(environ.losses)

In [ ]:
tmp = environ.get_loss_dict()
print(tmp.keys())
pp.pprint(tmp)

In [ ]:
print(opt['diff_sparsity_weights'])
print(opt['is_sharing'])
print(opt['diff_sparsity_weights'] and not opt['is_sharing'])
print(environ.opt['train']['Lambda_sharing'])
print(opt['train']['Lambda_sharing'])
print(environ.opt['train']['Lambda_sparsity'])
print(opt['train']['Lambda_sparsity'])
print(environ.opt['train']['policy_lr'])
print(opt['train']['policy_lr'])

### Policy / Logit stuff

In [ ]:
from scipy.special          import softmax

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=8,linewidth=132)

#### `get_task_logits(n)` Get logits for task group n

In [ ]:
task_logits = environ.get_task_logits(1)
print(task_logits)

#### `get_arch_parameters()`: Get last used logits from network

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

#### `get_policy_logits()`:  Get Policy Logits - returns same as `get_arch_parameters()`

In [ ]:
logs = environ.get_policy_logits()
for i in logs:
    print(i, '\n')
# probs = softmax(logs, axis= -1)
# for i in probs:
#     print(i, '\n')

#### `get_policy_prob()` : Gets the softmax of the logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
for i in policy_softmaxs:
    print(i, '\n')

#### `get_sample_policy( hard_sampling = False)` : Calls test_sample_policy of network with random choices based on softmax of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
policies,logits = environ.get_sample_policy(hard_sampling = False)

for l, p, s in zip(logits, policies, policy_softmaxs) :
    for  l_row, p_row, s_row in zip(l, p, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### `get_sample_policy( hard_sampling = True)` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
hard_policies, logits = environ.get_sample_policy(hard_sampling = True)

for p,l,s in zip(hard_policies, logits, policy_softmaxs) :
    for  p_row, l_row, s_row in zip(p, l, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### Print

In [ ]:
print(f" Layer    task 1      task 2      task 3")
print(f" -----    ------      ------      ------")
for idx, (l1, l2, l3) in enumerate(zip(hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1}       {l2}       {l3}")
    

    print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
def display_trained_policy(iter):

    policy_softmaxs = environ.get_policy_prob()
    policy_argmaxs = 1-np.argmax(policy_softmaxs, axis = -1)
    print(f"  Trained polcies at iteration: {iter} ")
    print(f"                   task 1                           task 2                         task 3        ")
    print(f" Layer       softmax        select          softmax        select          softmax        select   ")
    print(f" -----    ---------------   ------       ---------------   ------       ---------------   ------   ")
    for idx, (l1,l2,l3,  p1,p2,p3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2], policy_argmaxs[0], policy_argmaxs[1], policy_argmaxs[2]),1):
        print(f"   {idx}      {l1[0]:.4f}   {l1[1]:.4f}   {p1:4d}    {l2[0]:11.4f}   {l2[1]:.4f}   {p2:4d}    {l3[0]:11.4f}   {l3[1]:.4f}   {p3:4d}")

    print()
# print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
display_trained_policy(5)

In [ ]:
print(f"                        POLICIES (SOFTMAX)                                       task 3          ")
print(f" Layer    task1              task2            task3 softmax         softmax         argmax         softmax         argmax   ")
print(f" -----    -------------     -------------     -------------   ------   ")
for idx, (l1,l2,l3, h1,h2,h3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2],hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1[0]:.4f} {l1[1]:.4f}     {l2[0]:.4f} {l2[1]:.4f}     {l3[0]:.4f} {l3[1]:.4f}    {h3}")
    
print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
# print(policy_softmaxs[2], np.argmax(1-policy_softmaxs[2], axis = -1))
print(policy_softmaxs, np.argmax(policy_softmaxs, axis = -1))

#### `get_current_logits()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
logits  = (environ.get_current_logits())
for i in logits:
    print(i ,'\n')

#### `get_current_policy()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
pols  = (environ.get_current_policy())

for i in pols:
    print(i ,'\n')

#### `gumbel_softmax()`  

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', floatmode = 'maxprec_equal')
torch.set_printoptions(precision=8,linewidth=132)

In [ ]:
print(environ.temp)
# tau = environ.temp
tau = 1
for i in range(3): 
    logits_tensor = torch.tensor(logits[0])
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=tau, hard=False).cpu().numpy() 

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=tau, hard=True).cpu().numpy()
    
    for l, gs, gh in zip(lgts, gumbel_soft, gumbel_hard):
        print(f"   {l}   \t {gs}            \t {gh}")
#     print(lgts)
#     print(gumbel_soft)
#     print(gumbel_hard)
    print()

In [ ]:
for lgts in logits:
    logits_tensor = torch.tensor(lgts)
    print(lgts)
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=1, hard=False)
    print(gumbel_soft)

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=1, hard=True)
    print(gumbel_hard)
    print()

In [ ]:
smax = scipy.special.softmax(logs, axis =1)
# smax = np.array( 
# [[0.46973792, 0.530262  ],
#  [0.45025694, 0.549743  ],
#  [0.4443086 , 0.5556915 ],
#  [0.4138397 , 0.58616036],
#  [0.4140113 , 0.5859887 ],
#  [0.42114905, 0.57885087]])

print(smax.shape)
print(smax)
print(smax[0])
print(smax[0].sum())
print(np.random.choice((1,0), p =smax[0]))

In [ ]:
logs = np.array(
[[0.33064184, 0.42053092],
 [0.3532089 , 0.52056104],
 [0.3888512 , 0.5680909 ],
 [0.42039296, 0.694217  ],
 [0.4519742 , 0.73311865],
 [0.48401102, 0.7522658 ]],
)